## Import Libraries

In [ ]:
from fastai import *
from fastai.vision import *
import timm
import cv2
from functools import partial
from sklearn.model_selection import StratifiedKFold

## Setting Random Seed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(24)

## Define Loader

In [ ]:
def _load_format(path, convert_mode, after_open)->Image:
    image = np.load(path)
    return Image(pil2tensor(image, np.float32).div_(255)) #return fastai Image format

vision.data.open_image = _load_format

In [ ]:
df = pd.read_csv("train.csv")
df['id_code'] = df['id_code'].map(lambda x: os.path.join("train_rc",'{}.npy'.format(x)))
df.head()

In [ ]:
submit = pd.read_csv("sample_submission.csv")
submit['id_code'] = submit['id_code'].map(lambda x: os.path.join("train_rc",'{}.npy'.format(x)))
submit.head()

In [ ]:
def predict(X, coef=[0.5, 1.5, 2.5, 3.5]):
    X_p = np.copy(X)
    for i, pred in enumerate(X_p):
        if pred < coef[0]:
            X_p[i] = 0
        elif pred >= coef[0] and pred < coef[1]:
            X_p[i] = 1
        elif pred >= coef[1] and pred < coef[2]:
            X_p[i] = 2
        elif pred >= coef[2] and pred < coef[3]:
            X_p[i] = 3
        else:
            X_p[i] = 4
    return np.int8(X_p) 

### Ranger Optimizer Taken from: https://github.com/lessw2020/Ranger-Deep-Learning-Optimizer

In [ ]:
import math
import torch
from torch.optim.optimizer import Optimizer, required
import itertools as it

class Ranger(Optimizer):
    
    def __init__(self, params, lr=1e-3, alpha=0.5, k=6, N_sma_threshhold=5, betas=(.95,0.999), eps=1e-8, weight_decay=0):
        #parameter checks
        if not 0.0 <= alpha <= 1.0:
            raise ValueError(f'Invalid slow update rate: {alpha}')
        if not 1 <= k:
            raise ValueError(f'Invalid lookahead steps: {k}')
        if not lr > 0:
            raise ValueError(f'Invalid Learning Rate: {lr}')
        if not eps > 0:
            raise ValueError(f'Invalid eps: {eps}')
        
        #parameter comments:
        # beta1 (momentum) of .95 seems to work better than .90...
        #N_sma_threshold of 5 seems better in testing than 4.
        #In both cases, worth testing on your dataset (.90 vs .95, 4 vs 5) to make sure which works best for you.
        
        #prep defaults and init torch.optim base
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        super().__init__(params,defaults)
        
        #adjustable threshold
        self.N_sma_threshhold = N_sma_threshhold
        
        #now we can get to work...
        for group in self.param_groups:
            group["step_counter"] = 0
            #print("group step counter init")
                      
        #look ahead params
        self.alpha = alpha
        self.k = k 
        
        #radam buffer for state
        self.radam_buffer = [[None,None,None] for ind in range(10)]
        
        #lookahead weights
        self.slow_weights = [[p.clone().detach() for p in group['params']]
                                for group in self.param_groups]
        
        #don't use grad for lookahead weights
        for w in it.chain(*self.slow_weights):
            w.requires_grad = False
        
    def __setstate__(self, state):
        print("set state called")
        super(Ranger, self).__setstate__(state)
       
        
    def step(self, closure=None):
        loss = None
        #note - below is commented out b/c I have other work that passes back the loss as a float, and thus not a callable closure.  
        #Uncomment if you need to use the actual closure...
        
        #if closure is not None:
            #loss = closure()
            
        #------------ radam
        for group in self.param_groups:
    
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')
    
                p_data_fp32 = p.data.float()
    
                state = self.state[p]
    
                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)
    
                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']
    
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
    
                state['step'] += 1
                buffered = self.radam_buffer[int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma
                    if N_sma > self.N_sma_threshhold:
                        step_size = group['lr'] * math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        step_size = group['lr'] / (1 - beta1 ** state['step'])
                    buffered[2] = step_size
    
                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
    
                if N_sma > 4:
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size, exp_avg, denom)
                else:
                    p_data_fp32.add_(-step_size, exp_avg)
    
                p.data.copy_(p_data_fp32)
        
        
        #---------------- end radam step
        
        #look ahead tracking and updating if latest batch = k
        for group,slow_weights in zip(self.param_groups,self.slow_weights):
            group['step_counter'] += 1
            if group['step_counter'] % self.k != 0:
                continue
            for p,q in zip(group['params'],slow_weights):
                if p.grad is None:
                    continue
                q.data.add_(self.alpha,p.data - q.data)
                p.data.copy_(q.data)
            
        
            
        return loss


## Train 10 fold

In [ ]:
skf = StratifiedKFold(n_splits=10, shuffle = True, random_state=24)

In [ ]:
from sklearn.metrics import cohen_kappa_score, accuracy_score
def qk(y_pred, y):
    return torch.tensor(cohen_kappa_score(predict(y_pred), y, weights='quadratic'), device='cuda:0')
def accuracy(y_pred, y):
    return torch.tensor(accuracy_score(predict(y_pred), y), device='cuda:0')

In [ ]:
def weighted_mean_squared_error(pred:Tensor, targ:Tensor)->Rank0Tensor:
    "Mean squared error between `pred` and `targ`."
    pred,targ = flatten_check(pred,targ)
    mse=(pred-targ)** 2
    weights=torch.cuda.FloatTensor([ 2.028809,  9.897297,  3.665666, 18.974093, 12.413559])
    wmse=weights[targ[...,None].to(dtype=torch.int64)]*mse
    return wmse.mean()

In [ ]:
for train_index, val_index in skf.split(df.id_code, df['diagnosis']):
    opt=partial(Ranger)
    print(28*"====")
    sz=300
    bs=32
    tfms = get_transforms(do_flip=True,flip_vert=True, max_rotate=360, max_zoom=1.5, max_lighting=0.2)
    data_fold = (ImageList.from_df(df=df,path='',cols='id_code') 
        .split_by_idxs(train_index, val_index)
        .label_from_df(cols='diagnosis', label_cls=FloatList) 
        .transform(tfms,size=sz, padding_mode='border', resize_method=ResizeMethod.SQUISH)
        .databunch(bs=bs,num_workers=4) 
        .normalize(imagenet_stats)  
        )
    learn = cnn_learner(data_fold, timm.models.tf_efficientnet_b3, metrics=[qk, accuracy],callback_fns=ShowGraph).to_fp16()
    learn.unfreeze()
    learn.fit_one_cycle(10, max_lr=1e-6, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='valid_loss', name='b3_stage-1-'+str(i))])
    learn.load('b3_stage-1-'+str(i))
    learn.opt_func=opt
    learn.fit(20, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='valid_loss', name='b3_stage-2-1-'+str(i)),
                                      callbacks.CSVLogger(learn, filename = 'b3_stage-2-1-'+str(i)),
                                     ])
    learn.load('b3_stage-2-1-'+str(i))
    learn.freeze_to(-5)
    learn.fit(10, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='valid_loss', name='b3_stage-2-2-'+str(i)),
                                      callbacks.CSVLogger(learn, filename = 'b3_stage-2-2-'+str(i))
                                     ])
    learn.load('b3_stage-2-2-'+str(i))
    learn.freeze_to(-1)

    learn.fit(5, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='valid_loss', name='b3_stage-2-3-'+str(i)),
                                      callbacks.CSVLogger(learn, filename = 'b3_stage-2-3-'+str(i))
                                     ])
